In [ ]:
%pip install eyepop==3.12.0

### Setting authentication variables

In [ ]:
import getpass

EYEPOP_ACCOUNT_ID=input("Enter your Account UUID: ")
EYEPOP_API_KEY=getpass.getpass('Enter your API KEY: ')

### Setting a namespace prefix

In [ ]:
NAMESPACE_PREFIX="XXXXXX" # Add your namespace-prefix here

## Defining the abilities
This just defines the list of abilities as prototypes. The registration happens below.

In [ ]:
from eyepop import EyePopSdk
from eyepop.data.data_types import InferRuntimeConfig, VlmAbilityGroupCreate, VlmAbilityCreate, TransformInto
from eyepop.worker.worker_types import CropForward, ForwardComponent, FullForward, InferenceComponent, Pop
import json


ability_prototypes = [
    VlmAbilityCreate(
        name=f"{NAMESPACE_PREFIX}.image-classify.marketplace-presentation",
        description="Is the item ready to be shown on the marketplace",
        worker_release="qwen3-instruct",
        text_prompt="You are a Marketplace Presentation Curator. Your sole purpose is to judge the staging and environment of the photo. You only care if the photo looks professional and distraction-free. Instructions: Analyze the composition and background of the image using the following strict checklist. Fail Criteria (Output Fail_Presentation if ANY are true): 1. A person is wearing the item (model or selfie), or body parts (hands, feet, legs) are visible in the frame. 2. The image shows a pile of clothes, a "lot" of multiple items, or other garments are visible nearby (e.g., a closet floor). There must be only ONE item. 3. The background is cluttered with non-clothing objects (trash, power cords, furniture legs, unmade rumpled bedding, dirty laundry). 4.  The item is on a "loud" or busy patterned surface (e.g., complex oriental rug, multicolor quilt) that makes the item hard to focus on. 5. The item is the same color as the background (e.g., a black shirt on a black bedspread), causing the edges to disappear. 6. The item is draped over a chair, door handle, or balled up on a messy floor instead of being laid flat or hung neatly. Pass Criteria (Output Pass_Presentation if true): The item is the clear, singular focus. The background is relatively plain, clean, and contrasts enough with the item to show its shape. The item is laid flat centered in the image or hung on a hanger. Output: Return exactly one label: Pass_Presentation or Fail_Presentation",
        transform_into=TransformInto(
            classes=[
                "Pass_Presentation",
                "Fail_Presentation"
            ]
        ),
        config=InferRuntimeConfig(
            max_new_tokens=5,
            image_size=512
        ),
        is_public=False
    )
]



### Create all abilities, groups and aliases

In [ ]:
with EyePopSdk.dataEndpoint(api_key=EYEPOP_API_KEY, account_id=EYEPOP_ACCOUNT_ID) as endpoint:
    for ability_prototype in ability_prototypes:
        ability_group = endpoint.create_vlm_ability_group(VlmAbilityGroupCreate(
            name=ability_prototype.name,
            description=ability_prototype.description,
            default_alias_name=ability_prototype.name,
        ))
        ability = endpoint.create_vlm_ability(
            create=ability_prototype,
            vlm_ability_group_uuid=ability_group.uuid,
        )
        ability = endpoint.publish_vlm_ability(
            vlm_ability_uuid=ability.uuid,
            alias_name=ability_prototype.name,
        )
        ability = endpoint.add_vlm_ability_alias(
            vlm_ability_uuid=ability.uuid,
            alias_name=ability_prototype.name,
            tag_name="latest"
        )
        print(f"created ability {ability.uuid} with alias entries {ability.alias_entries}")

### Run individual inference via Pop

In [ ]:
example_urls = [
    "https://www.google.com/url?sa=i&url=https%3A%2F%2Fwww.pexels.com%2Fphoto%2Fshirt-on-hanger-on-branch-22441292%2F&psig=AOvVaw1BQP4QCl2KEeVFU_dWj91X&ust=1771021368538000&source=images&cd=vfe&opi=89978449&ved=0CBYQjRxqFwoTCLjevPv-1JIDFQAAAAAdAAAAABAE"
]

In [ ]:
pop = Pop(components=[
    InferenceComponent(
        ability=f"{NAMESPACE_PREFIX}.image-classify.marketplace-presentation:latest"
    )
])
with EyePopSdk.workerEndpoint(api_key=EYEPOP_API_KEY) as endpoint:
    endpoint.set_pop(pop)
    for url in example_urls:
        job = endpoint.load_from(url)
        while result := job.predict():
            print(json.dumps(result, indent=2))
    print("done")

### Evaluation

In [ ]:
from eyepop.data.data_types import EvaluateRequest

with EyePopSdk.dataEndpoint(api_key=EYEPOP_API_KEY, account_id=EYEPOP_ACCOUNT_ID) as endpoint:
    evaluate_req = EvaluateRequest(
        ability_uuid=ability.uuid,
        dataset_uuid="xxxxxxxxxxxxxxx" # Update with dataset UUID you would like to run the ability against

    )
    evaluation = endpoint.start_vlm_ability_evaluation(
            vlm_ability_uuid=ability.uuid,
            evaluate_request=evaluate_req
    )

    print(evaluation)
    print(evaluation.source)

In [ ]:
with EyePopSdk.dataEndpoint(api_key=EYEPOP_API_KEY, account_id=EYEPOP_ACCOUNT_ID) as endpoint:
    eval = endpoint.get_vlm_ability_evaluation(
      vlm_ability_uuid=ability.uuid,
      source=evaluation.source
    )
    print(eval)